In [9]:
import os
current_directory = os.getcwd()
print(f"Current Working Directory: {current_directory}")

from Tusc5ImageUtils import *
import numpy as np
from urllib.parse import urlparse
import matplotlib.pyplot as plt
import matplotlib as mpl
from skimage import exposure
mpl.rcParams['figure.dpi'] = 300
from cellpose import utils, io, plot, models, denoise
from scipy.ndimage import binary_erosion, binary_fill_holes, center_of_mass
from scipy.signal import find_peaks
import subprocess
import pandas as pd
import seaborn as sns
import re
import nd2
from skimage.measure import regionprops
from datetime import datetime

Current Working Directory: /Users/raphaeltinio/Lab Analysis MAC/ImageAnalysis


In [10]:
### Directory Configuration ###

'''
Input name of folder with .nd2 files in the parent directory of this python notebook
'''

parent_directory = os.path.dirname(current_directory)
target_folder_name = #INPUT

# Cell Fluoresence Retrieval

In [11]:
'''
.nd2 files with flipped top and bottom for whatever reason
'''
backward_files = ['3604R_GLUT1_WGA_0001.nd2',
                  '3604R_GLUT1_WGA_0003.nd2',
                  '3654L_GLUT1_WGA_0001.nd2',
                  '3654L_GLUT1_WGA_0003.nd2']

## 1. Set the min and max for the DAPI projections

In [12]:
'''
Code to set min and max for DAPI max projections

Automatically records min and max presets in file_mp_configurations.csv
If min and max were set multiple times, the most recent parameters are used
'''

if not parent_directory or not target_folder_name:
    raise ValueError('Make sure to input target folder name and parent directory.')

mp_configs_df = pd.read_csv('file_mp_configurations.csv')


target_folder_path = os.path.join(parent_directory, target_folder_name)
nd2_files = [f for f in os.listdir(target_folder_path) if f.endswith('.nd2')]

for nd2_file in nd2_files:

    # 1) Download Image Data
    print(f'Processing {nd2_file}')
    nd2_path = os.path.join(target_folder_path, nd2_file)

    f = nd2.ND2File(nd2_path)
    z_sep = f.voxel_size().z
    image = to_8bit(f.asarray())

    # Flip image if image is backwards
    if nd2_file in backward_files:
        image = np.flip(image, axis=0)

    # 2) DAPI max projection, Deblur, Segment
    DAPI_stack = image[:, 0, :, :].copy()

    z0, z1 = run_max_projector_app(DAPI_stack)
    mp_config = pd.DataFrame({
        'file_name': [nd2_file],
        'z0': [z0],
        'z1': [z1],
        'date': [datetime.now().strftime("%H:%M_%d_%m_%Y")]
    })
    mp_config.to_csv('file_mp_configurations.csv', mode='a', header=not pd.io.common.file_exists('file_mp_configurations.csv'), index=False)

Processing 3608R_GLUT1_WGA_0002.nd2


KeyboardInterrupt: 

## 2. Extract Traces

- This step takes a folder containing `.nd2` files and returns a DataFrame of traces.

In [13]:
### Setting paths ###
if not parent_directory or not target_folder_name:
    raise ValueError('Make sure to input target folder name and parent directory.')

target_folder_path = os.path.join(parent_directory, target_folder_name)


nd2_files = [f for f in os.listdir(target_folder_path) if f.endswith('.nd2')]

### Models ###
# Setting path for models
model_path_dapi = os.path.join(parent_directory, 'ImageAnalysis/cellpose_models/T5_DAPI_V4')
model_path_wga = os.path.join(parent_directory, 'ImageAnalysis/cellpose_models/T5_WGA_V2')

# Seting the DAPI (deblur) and WGA models
deblur_model = denoise.CellposeDenoiseModel(gpu=True, model_type= model_path_dapi, restore_type="deblur_cyto3")
wga_model = models.CellposeModel(gpu=True, pretrained_model=model_path_wga)

### Reading respective DAPI min and max each respective image stack ###
mp_configs_df = pd.read_csv('file_mp_configurations.csv')


all_data = pd.DataFrame()
unid_counter = 0

for nd2_file in nd2_files:

    # 1) Download Image Data
    print(f'Processing {nd2_file}')
    nd2_path = os.path.join(target_folder_path, nd2_file)

    f = nd2.ND2File(nd2_path)
    z_sep = f.voxel_size().z
    image = to_8bit(f.asarray())
    f.close()

    # Flip image if image is backwards
    if nd2_file in backward_files:
        image = np.flip(image, axis=0)

    # 2) DAPI max projection, Deblur, Segment
    DAPI_stack = image[:, 0, :, :].copy()

    if nd2_file in mp_configs_df['file_name'].values:
        file_mp_df = mp_configs_df.loc[mp_configs_df['file_name'] == nd2_file].iloc[-1]
        
        z0, z1 = file_mp_df['z0'], file_mp_df['z1']


    else:
        z0, z1 = run_max_projector_app(DAPI_stack)
        mp_config = pd.DataFrame({
            'file_name': [nd2_file],
            'z0': [z0],
            'z1': [z1],
            'date': [datetime.now().strftime("%H:%M_%d_%m_%Y")]
        })
        mp_config.to_csv('file_mp_configurations.csv', mode='a', header=not pd.io.common.file_exists('file_mp_configurations.csv'), index=False)


    mp_DAPI = max_proj(DAPI_stack[z0:z1].copy())

    # 3) Deblur and Segment DAPI max projection
    DAPI_masks, flows, styles, image_deblurred = deblur_model.eval(auto_brightness_contrast(mp_DAPI), diameter=None, channels=[0, 0])
    image_deblurred = image_deblurred[:, :, 0]  # resulting image has one channel, but it still needs to be indexed


    # DAPI filtering and eGFP identification
    coords_3d = nuclei_centers_of_mass(DAPI_stack, DAPI_masks)
    filtered_coords_3d, filtered_idxs = remove_outliers_local(coords_3d, num_closest_points=15, z_threshold=2)
    filtered_DAPI_masks = extract_masks(DAPI_masks, filtered_idxs)
    DAPI_masks = filtered_DAPI_masks.copy()

    ## Quick view
    ## plt.imshow(plot.mask_overlay(to_8bit(image_deblurred), DAPI_masks))
    ## plt.axis('off')
    ## plt.show()

    #Identifying cells in rip
    ##coords_2d = [(i[0], i[1]) for i in filtered_coords_3d]
    ##in_rip_dict = rip_identifier(nd2_file, image, DAPI_masks, coords_2d)

    # List for eGFP identification later
    eGFP_fluorescence_list = []
    
    # Initialize a list to accumulate cell data
    file_data_list = []

    '''
    Indiviudal Cell
    '''

    # 5) Segmentation of WGA channel
    mask_idxs = np.delete(np.unique(DAPI_masks), 0) - 1

    total_masks = len(mask_idxs)  # Total number of masks to process
    masks_found = 0  # Counter for the number of masks found

    for mask_id in mask_idxs:

        single_mask = extract_masks(DAPI_masks, mask_id)
        diam = get_mask_diameter(single_mask)
        expansion = 50

        sq_stacks = get_sq_stacks(image, single_mask)

        # Running the model of the expanded squares
        expanded_sq_WGA, z_level = extract_square_proj_expand(image, single_mask, expansion)

        expanded_mask, flows, styles = wga_model.eval(expanded_sq_WGA, diameter=diam, channels=[0, 0])

        # Removing 0-pixel boundary and finding the largest mask in the array
        WGA_mask = remove_boundary(expanded_mask, expansion)

        if len(np.unique(WGA_mask)) == 1:
            continue

        elif len(np.unique(WGA_mask)) > 2:
            WGA_mask = closest_mask_2d(single_mask, WGA_mask)

        masks_found += 1  # Increment the masks found counter

        # Z-axis profile
        trace_results = get_traces(sq_stacks, WGA_mask)

        # eGFP extraction
        eGFP_sum = np.sum(sq_stacks[1, z_level, :, :][(WGA_mask.astype(bool))])
        eGFP_sum_per_area = eGFP_sum / np.sum(WGA_mask)

        eGFP_fluorescence_list.append((mask_id, eGFP_sum_per_area))

        '''
        Organizing data
        '''

        # 6) Converting trace results into a pd dataframe
        cell_data = organize_data(trace_results, mask_id)

        # 7) Adding file information
        djid, eye, file_base = extract_information(nd2_file)

        nested_array = np.array(range(image.shape[0])) * z_sep
        cell_data['X_vals'] = [nested_array for i in range(len(cell_data))]
        cell_data['file_name'] = file_base
        cell_data['DJID'] = djid
        cell_data['Eye'] = eye
        cell_data['eGFP_Value'] = False
        cell_data['eGFP_Raw_Intensity'] = eGFP_sum_per_area

        # Adding in_rip information
        cell_data['in_rip'] = False

        ##for file_name, mask_ids in in_rip_dict.items():
        ##    if file_name == file_base and mask_id in mask_ids:
        ##        cell_data['in_rip'] = True

        # Accumulating cell data
        file_data_list.append(cell_data)

    # Print the number of masks found out of the total possible masks
    print(f'- Masks found: {masks_found}/{total_masks}')

    # Concatenating the list into a single DataFrame for the file
    file_data = pd.concat(file_data_list, ignore_index=True)

    # Processing eGFP data for the entire file
    eGFP_idxs = np.array([i[0] for i in eGFP_fluorescence_list])
    eGFP_vals = np.array([i[1] for i in eGFP_fluorescence_list])

    # Normalizing eGFP values
    eGFP_vals_normal = normalize(eGFP_vals)

    # Setting cells above .2 as positive
    eGFP_pos_idxs = eGFP_idxs[eGFP_vals_normal > .2]
    file_data.loc[file_data['mask_id'].isin(eGFP_pos_idxs), 'eGFP_Value'] = True

    # Accumulating data at the all_data level
    all_data = pd.concat([all_data, file_data], ignore_index=True)

Processing 3608R_GLUT1_WGA_0002.nd2


/var/folders/sl/syqsj8q94yn7kbv8l50mtwdw0000gn/T/ipykernel_12103/148859537.py:32: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  f = nd2.ND2File(nd2_path)


- Masks found: 82/88
Processing 3608R_GLUT1_WGA_0001.nd2
- Masks found: 85/93
Processing 3655R_GLUT1_WGA_0001.nd2
- Masks found: 69/133
Processing 3607R_GLUT1_WGA_0003.nd2
- Masks found: 6/83
Processing 3607R_GLUT1_WGA_0002.nd2
- Masks found: 9/66
Processing 3657R_GLUT1_WGA_0001.nd2
- Masks found: 94/113
Processing 3606R_GLUT1_WGA_0001.nd2
- Masks found: 84/113
Processing 3607R_GLUT1_WGA_0001.nd2


- Masks found: 22/88
Processing 3653L_GLUT1_WGA_0003.nd2
- Masks found: 90/126
Processing 3653L_GLUT1_WGA_0002.nd2


- Masks found: 86/112
Processing 3653L_GLUT1_WGA_0001.nd2
- Masks found: 55/104
Processing 3653L_GLUT1_WGA_0005.nd2


- Masks found: 61/94
Processing 3653L_GLUT1_WGA_0004.nd2


- Masks found: 39/106


# Exporting Data

In [33]:
export_df = all_data.copy()

In [31]:
## Code for quick examination of any single cell
##export_df_temp = export_df.copy()
##export_df_temp['Cell_unid'] = export_df.groupby(['file_name', 'mask_id']).ngroup()
##plot_single_cell(export_df_temp.query('Cell_unid == 10'), prominence =15, distance = 20)

In [34]:
export_df['X_vals'] = export_df['X_vals'].apply(lambda x: ', '.join(map(str, x)))
export_df['Y_vals'] = export_df['Y_vals'].apply(lambda x: ', '.join(map(str, x)))
export_df = export_df.loc[export_df['file_name'] != '3607R_GLUT1_WGA_0001']

### VVV ATTENTION VVV

raw_data_csv_name = #INPUT
export_df.to_csv('raw_data_folder/' + raw_data_csv_name + '.csv', index = False) # Carefully modify export csv name